    1. Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
    Также должны поддерживаться следующие операции:
•	извлечение элемента по индексу;
•	устанавка нового значения по индексу;
•	получение текущей длины массива.

    1.1 Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print

In [1]:
class Dynamic_Array():
    def __init__(self):
        self.array=[]
    def append(self,elem):
        return self.array.append(elem)
    def elem_by_index(self,index):
        return self.array[index]
    def new_value(self,index,value):
        self.array[index]=value
    def __len__(self):
        return len(self.array)
    def delete_by_ind(self,index):
        del self.array[index]
    def __str__(self):
        return str(self.array)

In [2]:
array=Dynamic_Array()
array.append(23)
array.append(11)
array.append(42)
array.append(69)
print(array)

[23, 11, 42, 69]


In [3]:
print(array.elem_by_index(3))

69


In [4]:
array.new_value(0,123)
print(array)

[123, 11, 42, 69]


In [5]:
print(len(array))

4


In [7]:
array.delete_by_ind(3)
print(array)

[123, 11, 42]


2. Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.


In [9]:
from collections import deque
def genchik(q1, q2):
    while len(q1) > 0 or len(q2) > 0:
        if len(q1) > 0:
            yield q1.pop()
        if len(q2) > 0:
            yield q2.pop()
            
q1 = deque([5, 23, 1, 8])
q2 = deque(['hh', 'cz', 'ru'])

gen = genchik(q1, q2)
for value in gen:
    print(value)

8
ru
1
cz
23
hh
5


3. Реализовать классы с медодом action():
а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь 
(очередь передается в конструктор).


b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.

In [12]:
from collections import deque

class Pump:
    def __init__(self, queue):
        self.queue = queue
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queue.append(value)
        except StopIteration:
            pass

class MultiAction:
    def __init__(self, obj, n):
        self.obj = obj
        self.n = n
    
    def action(self, gen):
        for _ in range(self.n):
            self.obj.action(gen)

class MultiPump:
    def __init__(self, queues):
        self.queues = queues
        self.queue_index = 0
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queues[self.queue_index].append(value)
            self.queue_index = (self.queue_index + 1) % len(self.queues)
        except StopIteration:
            pass

def www():
    for i in range(1, 8):
        yield i

q1 = deque()
q2 = deque()
q3 = deque()

pump1 = Pump(q1)
pump2 = Pump(q2)
multi_pump = MultiPump([q1, q2, q3])

gen = www()
pump1.action(gen)
pump2.action(gen)

multi_act = MultiAction(pump1, 3)
multi_act.action(gen)

multi_pump.action(gen)
multi_pump.action(gen)
multi_pump.action(gen)
multi_pump.action(gen)

print(q1)
print(q2)
print(q3)

deque([1, 3, 4, 5, 6])
deque([2, 7])
deque([])


4. При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).

In [14]:
import random
from collections import deque

class GenFromQ:
    def __init__(self, queues):
        self.queues = queues
    
    def action(self, output_queue):
        for queue in self.queues:
            while queue:
                output_queue.append(queue.popleft())

class Pump:
    def __init__(self, queue):
        self.queue = queue
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queue.append(value)
        except StopIteration:
            pass

# Генератор случайных сообщений
def message_generator():
    messages = ["Important message", "Urgent message", "Normal message"]
    while True:
        yield random.choice(messages)

# Создаем три очереди
queue1 = deque()
queue2 = deque()
queue3 = deque()
output_queue = deque()

# Создаем экземпляры классов Pump для каждой очереди
pump1 = Pump(queue1)
pump2 = Pump(queue2)
pump3 = Pump(queue3)

# Создаем экземпляр класса GenFromQ для объединения сообщений из разных очередей
gen_from_q = GenFromQ([queue1, queue2, queue3])

# Создаем генератор сообщений
gen = message_generator()

# Загружаем сообщения в очереди
pump1.action(gen)
pump2.action(gen)
pump3.action(gen)

# Объединяем сообщения из разных очередей в одну
gen_from_q.action(output_queue)

# Выводим сообщения на экран
while output_queue:
    print(output_queue.popleft())

Important message
Normal message
Normal message
